In [1]:
#to install R :
#conda install -c r r-irkernel
# to install a library from R
#!pip install library edgeR

In [2]:
#we just do it once
#!pip3 install xgboost

In [3]:
# a pachage that might need for some normalisation later on.
#!pip install rnanorm;
#import rnanorm

In [4]:
#this package has many of the normalisation procedures.
#!pip install bioinfokit
#!pip install bioinfokit --upgrade 

In [1]:
#!pip install conorm
from bioinfokit.analys import norm, get_data
import bioinfokit
import conorm # for tmm normalisation

In [2]:
from xgboost import XGBClassifier
import numpy as np
import pandas as pd
from pathlib import Path
import functools
# sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_recall_curve

import matplotlib.pyplot as plt

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import class_weight
from sklearn.feature_selection import SelectFromModel


In [3]:
#loading the counts data

path1=Path("/home/znazari/machine_learning_ppmi")

quant_rna1=pd.read_csv(path1/"p_ordered_bl_matrix_ir2_count.txt", header=None,index_col=[0] )
quant_rna=quant_rna1.rename(columns=quant_rna1.iloc[0]).drop(quant_rna1.index[0]).reset_index(drop=True)

In [4]:
#great idea to set the geneid as an index counter so to avoid its confliction with the rest.
dff1 = quant_rna.set_index('Geneid')

In [5]:
dff1

,10874,12499,12593,13039,13424,14281,14331,14426,15761,16580,...,4136,41381,4139,60002,60015,60024,60036,60095,60171,85242
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000186092,0,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ENSG00000187634,19,1,59,4,5,61,2,1,44,3,...,43,8,16,29,1,1,3,0,1,1
ENSG00000188976,384,649,529,347,440,383,302,324,336,234,...,775,369,321,292,344,342,858,280,161,1082
ENSG00000187961,115,158,164,80,171,119,99,133,122,44,...,275,121,101,106,109,105,205,85,71,310
ENSG00000187583,24,21,45,12,16,33,8,5,47,11,...,71,12,32,27,13,7,14,8,6,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000212907,385,369,60,138,460,138,173,144,151,31,...,379,257,102,148,176,305,507,115,140,881
ENSG00000198886,976,1577,379,440,1508,627,534,406,603,168,...,1403,652,333,546,669,1231,1727,480,474,2173
ENSG00000198786,511,763,234,222,1007,429,203,186,428,83,...,689,479,230,237,311,525,848,275,217,1838


In below we do the following steps:

1- we filter the rows in which their average is less than 5.

2- we apply tmm normalisation

3- we take log2 of the data

In [6]:
#finding average of each row, and making a new column for the average
i_average=dff1.mean(axis=1)
i_average
dff1["average"]=i_average

In [7]:
#we filter all the rows that their average is less than 5,then we drop the nan value rows
filter = dff1["average"]>5
# filtering data
dff1.where(filter, inplace = True)
dff_drop1=dff1.dropna()
dff_drop1

,10874,12499,12593,13039,13424,14281,14331,14426,15761,16580,...,41381,4139,60002,60015,60024,60036,60095,60171,85242,average
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634,19.0,1.0,59.0,4.0,5.0,61.0,2.0,1.0,44.0,3.0,...,8.0,16.0,29.0,1.0,1.0,3.0,0.0,1.0,1.0,14.421499
ENSG00000188976,384.0,649.0,529.0,347.0,440.0,383.0,302.0,324.0,336.0,234.0,...,369.0,321.0,292.0,344.0,342.0,858.0,280.0,161.0,1082.0,412.256011
ENSG00000187961,115.0,158.0,164.0,80.0,171.0,119.0,99.0,133.0,122.0,44.0,...,121.0,101.0,106.0,109.0,105.0,205.0,85.0,71.0,310.0,127.451202
ENSG00000187583,24.0,21.0,45.0,12.0,16.0,33.0,8.0,5.0,47.0,11.0,...,12.0,32.0,27.0,13.0,7.0,14.0,8.0,6.0,26.0,20.200849
ENSG00000187642,25.0,5.0,42.0,5.0,3.0,18.0,2.0,2.0,26.0,1.0,...,2.0,25.0,33.0,1.0,3.0,3.0,1.0,1.0,2.0,10.516266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000212907,385.0,369.0,60.0,138.0,460.0,138.0,173.0,144.0,151.0,31.0,...,257.0,102.0,148.0,176.0,305.0,507.0,115.0,140.0,881.0,216.925035
ENSG00000198886,976.0,1577.0,379.0,440.0,1508.0,627.0,534.0,406.0,603.0,168.0,...,652.0,333.0,546.0,669.0,1231.0,1727.0,480.0,474.0,2173.0,771.575672
ENSG00000198786,511.0,763.0,234.0,222.0,1007.0,429.0,203.0,186.0,428.0,83.0,...,479.0,230.0,237.0,311.0,525.0,848.0,275.0,217.0,1838.0,458.526167


In [8]:
dff_drop=dff_drop1.drop("average",axis='columns') 

In [9]:
dff_drop

,10874,12499,12593,13039,13424,14281,14331,14426,15761,16580,...,4136,41381,4139,60002,60015,60024,60036,60095,60171,85242
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634,19.0,1.0,59.0,4.0,5.0,61.0,2.0,1.0,44.0,3.0,...,43.0,8.0,16.0,29.0,1.0,1.0,3.0,0.0,1.0,1.0
ENSG00000188976,384.0,649.0,529.0,347.0,440.0,383.0,302.0,324.0,336.0,234.0,...,775.0,369.0,321.0,292.0,344.0,342.0,858.0,280.0,161.0,1082.0
ENSG00000187961,115.0,158.0,164.0,80.0,171.0,119.0,99.0,133.0,122.0,44.0,...,275.0,121.0,101.0,106.0,109.0,105.0,205.0,85.0,71.0,310.0
ENSG00000187583,24.0,21.0,45.0,12.0,16.0,33.0,8.0,5.0,47.0,11.0,...,71.0,12.0,32.0,27.0,13.0,7.0,14.0,8.0,6.0,26.0
ENSG00000187642,25.0,5.0,42.0,5.0,3.0,18.0,2.0,2.0,26.0,1.0,...,30.0,2.0,25.0,33.0,1.0,3.0,3.0,1.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000212907,385.0,369.0,60.0,138.0,460.0,138.0,173.0,144.0,151.0,31.0,...,379.0,257.0,102.0,148.0,176.0,305.0,507.0,115.0,140.0,881.0
ENSG00000198886,976.0,1577.0,379.0,440.0,1508.0,627.0,534.0,406.0,603.0,168.0,...,1403.0,652.0,333.0,546.0,669.0,1231.0,1727.0,480.0,474.0,2173.0
ENSG00000198786,511.0,763.0,234.0,222.0,1007.0,429.0,203.0,186.0,428.0,83.0,...,689.0,479.0,230.0,237.0,311.0,525.0,848.0,275.0,217.0,1838.0


In [10]:
# to add one so that later when we take log2 we do not end with conflict
dff=dff_drop.add(1)

In [11]:
#Trimmed Means of M-values (tmm)
dff_tmm = conorm.tmm(dff)
dff_tmm

,10874,12499,12593,13039,13424,14281,14331,14426,15761,16580,...,4136,41381,4139,60002,60015,60024,60036,60095,60171,85242
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634,18.663288,1.998614,45.990673,5.709843,5.856627,47.672041,3.251635,2.241096,37.632101,4.460318,...,34.439895,7.765007,16.323485,27.633633,2.334217,2.476657,3.936086,1.076306,2.051323,2.197449
ENSG00000188976,359.268287,649.549705,406.250944,397.405093,430.462052,295.259090,328.415180,364.178104,281.822625,262.043683,...,607.394515,319.228058,309.186008,269.888484,402.652458,424.746659,845.274409,302.442021,166.157168,1189.918846
ENSG00000187961,108.247068,158.889851,126.474351,92.499461,167.889961,92.268466,108.387848,150.153434,102.861077,50.178578,...,216.032070,105.258981,97.940909,98.559958,128.381943,131.262816,202.708415,92.562327,73.847630,341.703381
ENSG00000187583,23.329110,21.984759,35.259516,14.845593,16.593775,26.142732,9.754906,6.723288,40.140908,13.380954,...,56.356192,11.216121,31.686765,25.791391,16.339520,9.906628,14.760321,9.686755,7.179631,29.665567
ENSG00000187642,24.262274,5.995843,32.959982,6.851812,3.904418,14.609174,3.251635,3.361644,22.579261,2.230159,...,24.264472,2.588336,24.965330,31.318118,2.334217,4.953314,3.936086,2.152612,2.051323,3.296174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000212907,360.201451,369.743678,46.757184,158.733643,449.984141,106.877639,188.594856,162.479462,127.112875,35.682544,...,297.435458,222.596862,98.901114,137.247045,206.578218,378.928506,499.882887,124.851510,144.618276,969.075182
ENSG00000198886,911.701601,1576.906823,291.274262,503.608178,1472.941580,482.871637,579.874988,456.063041,505.106426,188.448436,...,1098.945746,563.394383,320.708468,503.853245,781.962745,1525.620653,1700.389033,517.703246,487.189227,2388.627490
ENSG00000198786,477.780164,763.470731,180.130136,254.659011,983.913262,330.628669,221.111210,209.542478,358.759366,93.666678,...,540.080174,414.133697,221.807354,219.226823,364.137875,651.360766,835.434195,297.060490,223.594214,2020.554717


In [12]:
#log2 normalisation
log_tmm=np.log2(dff_tmm)
data_log_tmm=pd.DataFrame(log_tmm)
data_log_tmm

,10874,12499,12593,13039,13424,14281,14331,14426,15761,16580,...,4136,41381,4139,60002,60015,60024,60036,60095,60171,85242
Geneid,,,,,,,,,,,,,,,,,,,,,
ENSG00000187634,4.222131,0.999000,5.523269,2.513451,2.550070,5.575071,1.701166,1.164204,5.233892,2.157147,...,5.106009,2.956987,4.028877,4.788353,1.222939,1.308394,1.976762,0.106088,1.036555,1.135830
ENSG00000188976,8.488918,9.343296,8.666227,8.634467,8.749742,8.205838,8.359377,8.508500,8.138644,8.033664,...,9.246490,8.318444,8.272331,8.076220,8.653391,8.730459,9.723276,8.240515,7.376405,10.216647
ENSG00000187961,6.758184,7.311883,6.982701,6.531373,7.391372,6.527766,6.760059,7.230294,6.684553,5.649000,...,7.755102,6.717800,6.613840,6.622930,7.004298,7.036314,7.663262,6.532353,6.206480,8.416601
ENSG00000187583,4.544059,4.458432,5.139941,3.891963,4.052570,4.708338,3.286128,2.749167,5.327001,3.742109,...,5.816502,3.487502,4.985808,4.688818,4.030294,3.308394,3.883652,3.276013,2.843910,4.890717
ENSG00000187642,4.600643,2.583963,5.042644,2.776486,1.965107,3.868803,1.701166,1.749167,4.496926,1.157147,...,4.600774,1.372025,4.641854,4.968926,1.222939,2.308394,1.976762,1.106088,1.036555,1.720792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000212907,8.492660,8.530382,5.547116,7.310464,8.813730,6.739816,7.559147,7.344114,6.989966,5.157147,...,8.216433,7.798289,6.627915,7.100631,7.690544,8.565782,8.965446,6.964069,7.176106,9.920465
ENSG00000198886,9.832418,10.622882,8.186234,8.976158,10.524484,8.915496,9.179598,8.833089,8.980444,7.558026,...,10.101904,9.138001,8.325119,8.976860,9.610956,10.575181,10.731649,9.015982,8.928338,11.221966
ENSG00000198786,8.900203,9.576429,7.492896,7.992423,9.942387,8.369068,7.788628,7.711099,8.486873,6.549464,...,9.077030,8.693953,7.793163,7.776281,8.508341,9.347313,9.706382,8.214613,7.804739,10.980536


In [13]:
#the first step to get the cpm norm is these two lines
#nm = norm()
#nm.cpm(df=dff)
#here is the cpm norm taking step 
#cpm_df = nm.cpm_norm
#cpm_df.head(5)

In [14]:
#loading the quant diagnosis: prediction

quant_predict1=pd.read_csv(path1/"the_diagnosis_count_ir2.txt", header=None ,index_col=[0])
quant_predict=quant_predict1.rename(columns=quant_predict1.iloc[0]).drop(quant_predict1.index[0]).reset_index(drop=True)

In [15]:
#transposed matrix
t_quant=data_log_tmm.T
t_quant

Geneid,ENSG00000187634,ENSG00000188976,ENSG00000187961,ENSG00000187583,ENSG00000187642,ENSG00000188290,ENSG00000187608,ENSG00000188157,ENSG00000131591,ENSG00000162571,...,ENSG00000198712,ENSG00000228253,ENSG00000198899,ENSG00000198938,ENSG00000198840,ENSG00000212907,ENSG00000198886,ENSG00000198786,ENSG00000198695,ENSG00000198727
10874,4.222131,8.488918,6.758184,4.544059,4.600643,6.987666,11.408988,6.807094,7.301083,3.707558,...,10.992960,5.600643,8.439362,12.715787,10.115736,8.492660,9.832418,8.900203,8.338995,9.483286
12499,0.999000,9.343296,7.311883,4.458432,2.583963,6.856981,10.004625,6.917863,8.242174,1.999000,...,11.493856,5.246928,9.316413,13.155873,10.055638,8.530382,10.622882,9.576429,7.158872,10.556464
12593,5.523269,8.666227,6.982701,5.139941,5.042644,7.461869,9.894828,7.155538,7.703842,4.825832,...,9.102208,4.423734,6.786304,11.201811,7.801254,5.547116,8.186234,7.492896,7.025770,8.281715
13039,2.513451,8.634467,6.531373,3.891963,2.776486,9.249515,13.999180,8.030727,7.011702,2.513451,...,9.626151,4.946411,7.410692,11.572525,8.992423,7.310464,8.976158,7.992423,7.504406,8.761379
13424,2.550070,8.749742,7.391372,4.052570,1.965107,7.791656,10.439827,6.357425,7.778889,1.965107,...,10.928726,5.519696,9.028502,12.372375,9.252820,8.813730,10.524484,9.942387,7.084048,10.328147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60024,1.308394,8.730459,7.036314,3.308394,2.308394,6.395857,10.465741,6.191037,7.498219,2.630322,...,10.827047,5.665946,9.153884,13.097520,10.224273,8.565782,10.575181,9.347313,6.751338,10.344568
60036,1.976762,9.723276,7.663262,3.883652,1.976762,7.576674,11.052909,7.508143,8.666760,3.146687,...,11.401978,5.784117,9.186215,13.050568,9.943988,8.965446,10.731649,9.706382,7.032044,10.277114
60095,0.106088,8.240515,6.532353,3.276013,1.106088,6.964069,9.391491,5.276013,7.139511,1.106088,...,9.573694,4.691051,7.660677,11.577256,8.799575,6.964069,9.015982,8.214613,6.012979,9.057373
60171,1.036555,7.376405,6.206480,2.843910,1.036555,6.186302,9.600704,3.943445,6.445946,1.036555,...,9.579587,4.284482,6.882045,11.642960,8.891423,7.176106,8.928338,7.804739,7.196426,8.691191


In [16]:
searchfor = ['Healthy Control', "Parkinson's Disease"]


In [17]:
#we just pick those that are either health control or parkinson diseases
quant_p=quant_predict[quant_predict['COHORT_DEFINITION'].astype(str).str.contains('|'.join(searchfor))]

In [18]:
quant_new_predict=quant_p['BL_PATNO']

In [19]:
r_quant_patients=[int(x) for x in quant_new_predict]

In [20]:
#here we select those rna-seq data that includes both health control and parkinson 
selected_quant_rna=data_log_tmm[r_quant_patients]

In [21]:
select_quant_rna=selected_quant_rna.T

In [22]:
quant_p['COHORT_DEFINITION'].replace(['Healthy Control', "Parkinson's Disease"],
                        [0, 1], inplace=True)

/tmp/ipykernel_6388/93247712.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  quant_p['COHORT_DEFINITION'].replace(['Healthy Control', "Parkinson's Disease"],


In [23]:
#here we make the model
bst_q = XGBClassifier(n_estimators=100, max_depth=30, learning_rate=0.2, objective='reg:logistic')


In [24]:
%%time
#here we repeat the machine learning procedure for k times XGboost, logistic regression!

k=5
g0=[]
p0=[]
for i in range(k):
    X_trainq, X_testq, y_trainq, y_testq = train_test_split(select_quant_rna,quant_p['COHORT_DEFINITION'], test_size=.3)
    logr=bst_q.fit(X_trainq, y_trainq)
    accuracy = accuracy_score(y_testq, logr.predict(X_testq))
    roc_score = roc_auc_score(y_testq,logr.predict(X_testq))
    g= (accuracy * 100.0)
    p= (roc_score*100.0)
    g0+=[g]
    p0+=[p]
aa1=round(sum(g0)/k)
aa2=round(sum(p0)/k)
aa3=round(np.std(g0))
aa4=round(np.std(p0))
print("average accuracy:",aa1)
print("average roc_auc score:",aa2)
print("accuracy std:",aa3)
print("auc_roc score std:",aa4)

         


average accuracy: 64
average roc_auc score: 52
accuracy std: 3
auc_roc score std: 1
CPU times: user 15min 2s, sys: 11 s, total: 15min 13s
Wall time: 7min 47s


In [25]:
fl=[]
for i in range(5):
    fl+=[i]
print("ki:",fl)
    

ki: [0, 1, 2, 3, 4]


In [26]:
%%time
model_RF = RandomForestClassifier(n_estimators=150)

# here we also make the code balanced with the provided data.

k=2
g0=[]
m0=[]

for i in range(k):
    X_trainq, X_testq, y_trainq, y_testq = train_test_split(select_quant_rna,quant_p['COHORT_DEFINITION'], test_size=.3)
    classes_weights = class_weight.compute_sample_weight(class_weight='balanced',y=y_trainq)
    model_RF.fit(X_trainq, y_trainq,sample_weight=classes_weights )
    preds_q = model_RF.predict(X_testq)
    accuracy = accuracy_score(y_testq, preds_q)
    roc_score=roc_auc_score(y_testq,preds_q)
    g= (accuracy * 100.0)
    m= (roc_score * 100.0)
    g0+=[g]
    m0+=[m]
cc1=round(sum(g0)/k,2)
cc2=round(sum(m0)/k,2)
print("average accuracy:",cc1)
print("average roc_auc score:",cc2)


average accuracy: 63.84
average roc_auc score: 49.73
CPU times: user 8.51 s, sys: 257 ms, total: 8.77 s
Wall time: 8.93 s


# Selecting Features

In [27]:

sel = SelectFromModel(model_RF)
sel.fit(X_trainq, y_trainq)
selected_feat= X_trainq.columns[(sel.get_support())]
selected_feat

Index(['ENSG00000188976', 'ENSG00000187961', 'ENSG00000187583',
       'ENSG00000188290', 'ENSG00000188157', 'ENSG00000162571',
       'ENSG00000184163', 'ENSG00000169962', 'ENSG00000160072',
       'ENSG00000205090',
       ...
       'ENSG00000154620', 'ENSG00000165246', 'ENSG00000012817',
       'ENSG00000198888', 'ENSG00000198763', 'ENSG00000198712',
       'ENSG00000198938', 'ENSG00000212907', 'ENSG00000198695',
       'ENSG00000198727'],
      dtype='object', name='Geneid', length=4124)

In [28]:
coeff = {}
for idx, item in enumerate(model_RF.best_estimator_.named_steps["classifier"].feature_importances_):
    coeff[new_cols[idx]] = item
for key in sorted(coeff,key=lambda x: abs(coeff[x]),reverse=True):
    print("{0:40}:{1}".format(key,coeff[key])) 

AttributeError: 'RandomForestClassifier' object has no attribute 'best_estimator_'

# Hyperparameter tuning for Random forest 

In [ ]:
n_estimators=[int(x) for x in np.linspace(start=5, stop=70, num=10)]
max_depth=[2, 4, 6, 10, 12, 18, 20]
min_samples_split=[2,5]
max_features=['sqrt']
min_samples_leaf=[1,2]
bootstrap=[False]

In [ ]:
#creating the param grid
param_grid={'n_estimators':n_estimators,
           'max_depth':max_depth,
           'min_samples_split':min_samples_split,
           'max_features':max_features,
           'min_samples_leaf':min_samples_leaf,
           'bootstrap':bootstrap}
print(param_grid)

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
mdl_RF = RandomForestClassifier()


In [ ]:
rf_Grid = GridSearchCV(estimator = mdl_RF, param_grid = param_grid,
                      cv = 3, verbose = 2, n_jobs = 4)

In [ ]:
%%time
rf_Grid.fit(X_trainq, y_trainq)

In [ ]:
X_trainq.isna()

In [ ]:
rf_Grid.best_params_

In [ ]:
hp_RF = RandomForestClassifier(bootstrap=False, max_depth=10, n_estimators=42, max_features='sqrt', min_samples_leaf=1, min_samples_split=2)


# check the data spiliting correctness

In [ ]:
print('Accuracy:  ', accuracy_score(y_testq,bst_q.predict(X_testq)))
print('Precision: ', precision_score(y_testq,bst_q.predict(X_testq)))
print('Recall:    ', recall_score(y_testq,bst_q.predict(X_testq)))
print('F1:        ', f1_score(y_testq,bst_q.predict(X_testq)))
print('ROC-AUC:   ', roc_auc_score(y_testq,bst_q.predict(X_testq)))
print('\nConfusion Matrix\n{0}'.format(confusion_matrix(y_testq,bst_q.predict(X_testq))))

In [ ]:
print('Accuracy:  ', accuracy_score(y_trainq,bst_q.predict(X_trainq)))
print('Precision: ', precision_score(y_trainq,bst_q.predict(X_trainq)))
print('Recall:    ', recall_score(y_trainq,bst_q.predict(X_trainq)))
print('F1:        ', f1_score(y_trainq,bst_q.predict(X_trainq)))
print('ROC-AUC:   ', roc_auc_score(y_trainq,bst_q.predict(X_trainq)))
print('\nConfusion Matrix\n{0}'.format(confusion_matrix(y_trainq,bst_q.predict(X_trainq))))

*confusion matrix* TN, FP; FN, TP

In [ ]:
roc = roc_curve(y_testq,bst_q.predict_proba(X_testq)[:,1])
pr  = precision_recall_curve(y_testq,bst_q.predict_proba(X_testq)[:,1])

f = plt.figure(figsize=(20,7))
ax = f.add_subplot(121)
ax.plot(roc[0],roc[1])
ax.set_ylabel('True Positive Rate')
ax.set_xlabel('False Positive Rate')
ax.set_title('ROC Curve')
ax.grid(which='both')
ax = f.add_subplot(122)
ax.plot(pr[1],pr[0])
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
ax.set_title('PR-curve')
ax.grid(which='both')
plt.show()

In [ ]:
roc = roc_curve(y_trainq,bst_q.predict_proba(X_trainq)[:,1])
pr  = precision_recall_curve(y_trainq,bst_q.predict_proba(X_trainq)[:,1])

f = plt.figure(figsize=(20,7))
ax = f.add_subplot(121)
ax.plot(roc[0],roc[1])
ax.set_ylabel('True Positive Rate')
ax.set_xlabel('False Positive Rate')
ax.set_title('ROC Curve')
ax.grid(which='both')
ax = f.add_subplot(122)
ax.plot(pr[1],pr[0])
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
ax.set_title('PR-curve')
ax.grid(which='both')
plt.show()

In [ ]:
roc = roc_curve(y_train_gb,m_gb.predict_proba(X_train_gb)[:,1])
pr  = precision_recall_curve(y_train_gb,m_gb.predict_proba(X_train_gb)[:,1])

f = plt.figure(figsize=(20,7))
ax = f.add_subplot(121)
ax.plot(roc[0],roc[1])
ax.set_ylabel('True Positive Rate')
ax.set_xlabel('False Positive Rate')
ax.set_title('ROC Curve')
ax.grid(which='both')
ax = f.add_subplot(122)
ax.plot(pr[1],pr[0])
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
ax.set_title('PR-curve')
ax.grid(which='both')
plt.show()